In [6]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files("zeelpatel19310068/cpcb-2023-dataset", path=".", unzip=True)

In [1]:
import git

repo = git.Repo(search_parent_directories=True)
repo

<git.repo.base.Repo '/home/patel_zeel/aqmsp/.git'>

'aqmsp_data'